### Name : Muhammad Safi (2303.khi.deg.023)
### Assignment Partners: Huzaifa Ali (2303.khi.deg.016)
### Shiekh Muhammad Sabih (2303.khi.deg.010)

In [1]:
from time import sleep

import pyspark
from IPython.display import clear_output, display
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json,udf,when,split
from pyspark.sql.streaming import StreamingQuery
from pyspark.sql.types import DateType, IntegerType, StringType, StructType

## Importing the dataset and naming the columns according to the dataset

In [2]:
spark = SparkSession.builder.appName("PySpark Assignment").getOrCreate()
titanic = spark.read.option("header", "true").option("inferSchema", "true").option("header", "false").csv("data/titanic.csv")
columns = ["PassengerId","Survived","Pclass","Name","Sex","Age","SibSp","Parch","Ticket","Fare","Cabin","Embarked","Timestamp"]

for i, col_name in enumerate(titanic.columns):
    titanic = titanic.withColumnRenamed(col_name, columns[i])
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)



### Modifying the Schema
Changing the data type of survived to string and coverting the binary values into string so that we can categorize it.

In [3]:
titanic = titanic.withColumn("Survived", when(col("Survived") == 0, "No").otherwise("Yes").cast("string"))
titanic.show()


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|          Timestamp|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|          1|      No|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|2020-01-01 13:45:25|
|          2|     Yes|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|2020-01-01 13:44:48|
|          3|     Yes|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|2020-01-01 13:38:11|
|          4|     Yes|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|2020-01-01 13:32:00|
|          5|      No|     3|Allen, Mr. Willia...|  mal

### Updated Schema

In [4]:
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: string (nullable = false)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)



In [5]:
titanic.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|          Timestamp|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|          1|      No|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|2020-01-01 13:45:25|
|          2|     Yes|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|2020-01-01 13:44:48|
|          3|     Yes|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|2020-01-01 13:38:11|
|          4|     Yes|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|2020-01-01 13:32:00|
|          5|      No|     3|Allen, Mr. Willia...|  mal

### Checking the minimum, maximum and mean values of the numerical data

In [6]:
numerical_cols = [col_name for col_name, col_type in titanic.dtypes if col_type in ['int', 'bigint', 'float', 'double']]
numerical_df = titanic.select(*numerical_cols)


min_max_mean=numerical_df.describe()
min_max_mean.select(numerical_cols).summary('min','max','mean').show()

+-------+------------------+------------------+------------------+------------------+-----------------+------------------+
|summary|       PassengerId|            Pclass|               Age|             SibSp|            Parch|              Fare|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+
|    min|                 1|0.8360712409770491|                 0|                 0|                0|               0.0|
|    max|               891|               891|                80|               891|              891|               891|
|   mean|497.27076840304596|179.62894264325715|167.64315089562422|180.12515025772694|179.6375301872114|297.04536731315113|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+



## Udf function to change the last letter of categorical data to 1

In [7]:
cat_columns = ["Sex","Cabin","Embarked","Survived"]

def change_last_letter_after_space(word):
    if word is not None:
        words = word.split()
        for i in range(len(words)):
            words[i] = words[i][:-1] + "1"
        return " ".join(words)
    return word
change_last_letter_udf = udf(change_last_letter_after_space, StringType())
for column in cat_columns:
    titanic = titanic.withColumn(column, change_last_letter_udf(titanic[column]))
titanic.show()
    

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|          Timestamp|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|          1|      N1|     3|Braund, Mr. Owen ...|  mal1|  22|    1|    0|       A/5 21171|   7.25| null|       1|2020-01-01 13:45:25|
|          2|     Ye1|     1|Cumings, Mrs. Joh...|femal1|  38|    1|    0|        PC 17599|71.2833|  C81|       1|2020-01-01 13:44:48|
|          3|     Ye1|     3|Heikkinen, Miss. ...|femal1|  26|    0|    0|STON/O2. 3101282|  7.925| null|       1|2020-01-01 13:38:11|
|          4|     Ye1|     1|Futrelle, Mrs. Ja...|femal1|  35|    1|    0|          113803|   53.1| C121|       1|2020-01-01 13:32:00|
|          5|      N1|     3|Allen, Mr. Willia...|  mal

## Sorting the DataFrame by its first column

In [8]:
sorted_titanic = titanic.orderBy(titanic.columns[0])
sorted_titanic.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|          Timestamp|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|          1|      N1|     3|Braund, Mr. Owen ...|  mal1|  22|    1|    0|       A/5 21171|   7.25| null|       1|2020-01-01 13:45:25|
|          2|     Ye1|     1|Cumings, Mrs. Joh...|femal1|  38|    1|    0|        PC 17599|71.2833|  C81|       1|2020-01-01 13:44:48|
|          3|     Ye1|     3|Heikkinen, Miss. ...|femal1|  26|    0|    0|STON/O2. 3101282|  7.925| null|       1|2020-01-01 13:38:11|
|          4|     Ye1|     1|Futrelle, Mrs. Ja...|femal1|  35|    1|    0|          113803|   53.1| C121|       1|2020-01-01 13:32:00|
|          5|      N1|     3|Allen, Mr. Willia...|  mal

## Saving the Resultant Dataset in parquet
this code overwrites each time the changes are made and the file is saved.

In [10]:
try:
    sorted_titanic.write.mode('overwrite').parquet("titanic_results.parquet")
except:
    print('Expection caught')